In [ ]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer

In [ ]:
model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )

In [ ]:
model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )

In [ ]:


 
# 3. クエリのエンコード

query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model.pretrained_model.device)

# 4. モデル応答の生成
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 200,
}
response_tensor = ppo_trainer.generate([item for item in query_tensor], return_prompt=False, **generation_kwargs)
response_txt = tokenizer.decode(response_tensor[0])
print(response_txt)

In [ ]:

# 2. トレーナーの初期化
ppo_config = {"batch_size": 1,
              "gradient_accumulation_steps":1,
              }
ppo_config = PPOConfig(
    model_name="llama",
    learning_rate=1.41e-6,
    mini_batch_size=1,
    gradient_accumulation_steps=1,
    batch_size=1,

)
#config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(ppo_config, model, model_ref, tokenizer)


In [ ]:

reward = [torch.tensor(.3, device=model.pretrained_model.device)]
reward = [torch.tensor(-.1, device=model.pretrained_model.device)]
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

# KTO

In [ ]:
model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model =AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )

model_ref =AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )

In [ ]:
from transformers import pipeline

question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"

def gen_prompt(q):
    return f"{question_template}{q}{answer_template}"
pipe=pipeline("text-generation",model=model,tokenizer=tokenizer,max_new_tokens=100)

q_list=[
"元気ですか?",
"ドラえもんの友達は誰ですか?",
"のび太の友達は誰ですか?",
]
for q in q_list:
    query_txt = gen_prompt(q)
    ans=pipe(query_txt)
    print(ans[0]["generated_text"][len(query_txt):])

In [ ]:
from datasets import load_dataset
from datasets import Dataset
kto_dataset_dict = {
    "prompt": [
        gen_prompt("ドラえもんの友達は誰ですか?"),
        gen_prompt("ドラえもんの友達は誰ですか?"),
        gen_prompt("元気ですか?"),
        gen_prompt("元気ですか?"),
    ],
    "completion": [
        "ドラえもんの友達は、のび太、ジャイアン、スネ夫、ドラミちゃんなどです",
        ' はい、確かに「友達」と呼ばれる人物がいます。その人物は「山田太郎」です。彼は、同じクラスの同級生であり、同じ部活にも所属しています。',
        "はい、元気です。ありがとうございます。",
        "毎日つらいです｡",
    ],
    "label": [
        True,
        False,
        True,
        False,
    ],
}
train_dataset=Dataset.from_dict(kto_dataset_dict)

In [ ]:
from trl import KTOConfig, KTOTrainer

training_args = KTOConfig(
    output_dir="test",
    beta=0.1,
    desirable_weight=1.0,
    undesirable_weight=1.0,
)

kto_trainer = KTOTrainer(
    model,
    model_ref,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

In [ ]:
kto_trainer.train()

# orpo

In [ ]:
model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model =AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )

model_ref =AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )

pipe=pipeline("text-generation",model=model,tokenizer=tokenizer,max_new_tokens=100)

In [ ]:
q_list=[
"元気ですか?",
"ドラえもんの友達は誰ですか?",
"のび太の友達は誰ですか?",
"クレヨンしんちゃんの友達は誰ですか?"
'質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。,"質問：電子機器で使用される最も主要な電子回路基板の事をなんと言う？\n選択肢：0.掲示板,1.パソコン,2.マザーボード,3.ハードディスク,4.まな板","質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。\n質問：電子機器で使用される最も主要な電子回路基板の事をなんと言う？\n選択肢：0.掲示板,1.パソコン,2.マザーボード,3.ハードディスク,4.まな板',
'質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。,"質問：明日の天気は?\n選択肢：0.あああ,1.パソコン,2.マザーボード,3.滝,4.雨","質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。\n質問：電子機器で使用される最も主要な電子回路基板の事をなんと言う？\n選択肢：0.掲示板,1.パソコン,2.マザーボード,3.ハードディスク,4.まな板',
]
for q in q_list:
    query_txt = gen_prompt(q)
    ans=pipe(query_txt)
    print(ans[0]["generated_text"][len(query_txt):])

In [ ]:

from trl import ORPOConfig, ORPOTrainer
from datasets import load_dataset
from datasets import Dataset
kto_dataset_dict = {
    "prompt": [
        gen_prompt("ドラえもんの友達は誰ですか?"),
        gen_prompt("元気ですか?"),
        gen_prompt('質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。,"質問：電子機器で使用される最も主要な電子回路基板の事をなんと言う？\n選択肢：0.掲示板,1.パソコン,2.マザーボード,3.ハードディスク,4.まな板","質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。\n質問：電子機器で使用される最も主要な電子回路基板の事をなんと言う？\n選択肢：0.掲示板,1.パソコン,2.マザーボード,3.ハードディスク,4.まな板'),
        gen_prompt('質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。,"質問：明日の天気は?\n選択肢：0.あああ,1.パソコン,2.マザーボード,3.滝,4.雨","質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。\n質問：電子機器で使用される最も主要な電子回路基板の事をなんと言う？\n選択肢：0.掲示板,1.パソコン,2.マザーボード,3.ハードディスク,4.まな板'),
    ],
    "chosen": [
        "ドラえもんの友達は、のび太、ジャイアン、スネ夫、ドラミちゃんなどです",
        "はい、元気です。ありがとうございます。",
        "2",
        "4",
    ],
    
    "rejected": [
        ' はい、確かに「友達」と呼ばれる人物がいます。その人物は「山田太郎」です。彼は、同じクラスの同級生であり、同じ部活にも所属しています。',
        "毎日つらいです｡",
        " 回答は、2.マザーボードです。",
        "2",
    ],
}
train_dataset=Dataset.from_dict(kto_dataset_dict)

orpo_config = ORPOConfig(
    output_dir="test",
    beta=0.1, # the lambda/alpha hyperparameter in the paper/code
    learning_rate=10**-7,
)

orpo_trainer = ORPOTrainer(
    model,
    args=orpo_config,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

In [ ]:

orpo_trainer.train()

# bumpo dataset

In [ ]:

model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model =AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )

model_ref =AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )

pipe=pipeline("text-generation",model=model,tokenizer=tokenizer,max_new_tokens=100)

In [ ]:
ds=load_dataset("hatakeyama-llm-team/BumpoRikai",split="train")
ds

In [ ]:
ds=ds.shuffle()
q_a_list=[]
for i in range(10):
    record=ds[i]
    q=gen_prompt(record["instruction"]+"\n"+record["question"])
    a=record["answer"]
    model_ans=pipe(q)[0]["generated_text"][len(q):]
    q_a_list.append((q,a,model_ans))
    #print(q,model_ans)
q_a_list

In [ ]:
tokenizer.encode(model_ans)

In [ ]:
pipe(q)[0]

In [ ]:
data_path="/home/hatakeyama/python/EvalPractice/3_finetune/data/all.jsonl.parquet"
ds=load_dataset("parquet",data_files=data_path,split="train")

In [ ]:
record=ds[8]
t=record["text"]
pos=t.find("応答:")
t=t[pos:pos+10]
tokenizer.encode(t),tokenizer.tokenize(t)

In [ ]:
tokenizer.encode(":")

In [ ]:

model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"
model_name="../../llm/models/hf/step41300"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model =AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )


In [ ]:

tokenizer.add_special_tokens({'additional_special_tokens': ['\n']})
pipe=pipeline("text-generation",model=model,tokenizer=tokenizer,max_new_tokens=100)
print(pipe(gen_prompt("こんにちは"))[0]["generated_text"])

In [ ]:
model_name="llm-jp/llm-jp-13b-v1.0"

model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
#tokenizer.add_tokens("\n")
#tokenizer.add_tokens(" ")
tokenizer.encode("a\na")

In [ ]:
id_to_w={k:v for v,k in tokenizer.vocab.items()}
id_to_w[8]

In [ ]:
# 変更したいトークンと新しいトークンの設定
old_token = '_'
new_token = ' '

# トークンのIDを取得
token_id = tokenizer.vocab[old_token]

# vocab と ids_to_tokens を更新
tokenizer.vocab[new_token] = token_id
#del tokenizer.vocab[old_token]
#tokenizer.ids_to_tokens[token_id] = new_token

In [ ]:
tokenizer.encode("aa\nbb")
tokenizer.encode("javascript\n#!fu")

In [ ]:
tokenizer.remove_token